# Database design
First we have created a database desing.<img src="design.png" />
# Creting import tables
All the scripts can be found <a href="https://github.com/TravisCG/summitimport">here</a>. Based on the design we created scripts to read the raw text files created by Eric and format it to a proper import table. The first four tables have no foreign keys, so you need to run it first.
## Antibody table
To create antibody table we have used the antibody.py script. The antibody table also contains information about the representation of the data in the website (colour, shapes, etc).

In [ ]:
python3 antibody.py ../../experiment_stats/All_stat_table_final.tbl ../../experiment_stats/Factors_shape_colour_final.tbl >../tables_txt/antibody.tsv

## Cell_lines table
Cell line names are a bit tricky, because the third part of the experiment name contains the cell line information. However some times it is unknown, sometimes it is set to 'cellline', sometimes it is use different case. Please check the output for potential problems!

Right now the import script converts everything to upper case and replace CELLLINE with UNKNOWN.

In [ ]:
python3 celllines.py ../../experiment_stats/All_stat_table_final.tbl >../tables_txt/cell_lines.tsv

## Genome table
This table is very simple. Right now contains only two entries. This script is just a place holder for future.

In [ ]:
python3 genome.py >../tables_txt/genome.tsv

## Consensus_motif table

In [ ]:
python3 consensus.py ../../motif_info/motif_info_final.tbl >../tables_txt/consensus_motif.tsv

## Experiment table
The creation of this table depends on the Anitbody, Cell_lines and Genome tables.

This script also replace 'CELLLINE' with 'UNKNOWN'.

In [ ]:
python3 experiment.py ../tables_txt/antibody.tsv ../tables_txt/cell_lines.tsv ../tables_txt/genome.tsv ../../experiment_stats/All_stat_table_final.tbl >../tables_txt/experiment.tsv

## Denovo_motif table
Right now one experiment has only one denovo motif, but later additional motifs can be added. This import script only works with 1-1 corrispondence.

In [ ]:
python3 denovo.py ../../experiment_stats/All_stat_table_final.tbl >../tables_txt/denovo_motif.tsv

## anti2cons table
This table is a connection between Antibody and Consensus_motif. Because there are experiments in the motif_info table, whose not in the experiment table, the script will skip some records.

In [ ]:
python3 anti2cons.py ../tables_txt/consensus_motif.tsv ../tables_txt/antibody.tsv ../../motif_info/motif_info_final.tbl >../tables_txt/anti2cons.tsv

## Average_deviation table
To create this table I need to read a lots of files. For some unknown reason I can see var2 and var3 files. For example: hs_TFAP2C-var.3_distancestat.tbl. Right now my script does not handle variations, so skip them.

In [ ]:
python3 avrdevi.py ../tables_txt/experiment.tsv ../tables_txt/consensus_motif.tsv ../../MedianAverage_dataTable/*.tbl >../tables_txt/average_deviation.tsv

## Motif_pos table
Creating motif_pos table is quite straight forward. The only questionable part is the trivial field in the table. Right now I calculate it from the motif start and motif end position.

The script skip var2 and var3 files.

In [ ]:
python3 motifpos.py ../tables_txt/consensus_motif.tsv ../../motif_vs_summit_distances/* >../tables_txt/motif_pos.tsv

## Peak table
If an experiment did not exists in the experiment table, the script will skip it.

In [ ]:
python3 peak.py ../tables_txt/experiment.tsv ../../homer_peakstext/* >../tables_txt/peak.tsv

## Summit table
This is the most complicated step, because the peak table is 5.6GB. This file is too big to store it in memory. It is a two step process.

In the first step we create an intermediate summit file using the experiment name and the motif positions.

In [ ]:
python3 summit.py ../tables_txt/consensus_motif.tsv ../tables_txt/motif_pos.tsv ../../motif_vs_summit_distances/* >../tables_txt/tmp1.tsv

The intermediate file (tmp1.tsv) is also quite big (11GB), so the next step process only one chromosome. It is use less memory and we can run it parallel. The first parameter is the chromosome name.

In [ ]:
python3 summit2.py 1 ../tables_txt/peak.tsv ../tables_txt/experiment.tsv ../tables_txt/tmp1.tsv >../tables_txt/summit.chr1.tsv &
python3 summit2.py 2 ../tables_txt/peak.tsv ../tables_txt/experiment.tsv ../tables_txt/tmp1.tsv >../tables_txt/summit.chr2.tsv &
python3 summit2.py 3 ../tables_txt/peak.tsv ../tables_txt/experiment.tsv ../tables_txt/tmp1.tsv >../tables_txt/summit.chr3.tsv &
python3 summit2.py 4 ../tables_txt/peak.tsv ../tables_txt/experiment.tsv ../tables_txt/tmp1.tsv >../tables_txt/summit.chr4.tsv &

There is a wrapper script which runs four chromosomes parallel (runallchr.sh). Finally we need to concatenate all the summit.chr\*.tsv files. There is no primary key, so the order of the files in the concatenation does not matter. The primary key is created by awk.

In [ ]:
cd ../tables_txt
cat summit.chr1.tsv summit.chr2.tsv summit.chr3.tsv summit.chr4.tsv summit.chr5.tsv summit.chr6.tsv summit.chr7.tsv summit.chr8.tsv summit.chr9.tsv summit.chr10.tsv summit.chr11.tsv summit.chr12.tsv summit.chr13.tsv summit.chr14.tsv summit.chr15.tsv summit.chr16.tsv summit.chr17.tsv summit.chr18.tsv summit.chr19.tsv summit.chr20.tsv summit.chr21.tsv summit.chr22.tsv summit.chrX.tsv summit.chrY.tsv | awk '{print NR"\t"$0}'>summit.tsv
rm summit.chr* tmp1.tsv

## Dbsnp table
All the snp information we need to find overlap between the SNPs and TFBSs.

In [ ]:
%%bash
wget ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh38p7/VCF/All_20180418.vcf.gz
zcat All_20180418.vcf.gz | awk '!/^#/{print $3"\t"$1"\t"$2"\t"$4"\t"$5}' >../tables_txt/dbsnp.tsv
rm All_20180418.vcf.gz

## Reference table
Reference table contains the reference sequence. The website use it to show the base pairs in dbsnp view. Right now there is no check about the genome table connection, so please set the correct genome id (in this example it is 1, but in the case of mouse, you need to set it to 2).

In [ ]:
%%bash
wget ftp://ftp.ensembl.org/pub/release-95/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
python3 ref.py Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz 1 >../tables_txt/reference.tsv
rm Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz

## Pfm table
This table contains the position frequency matrix for evey motifs.

In [ ]:
%%bash
python3 pfm.py ../tables_txt/consensus_motif.tsv ../../motif_info/matrices/meme >../tables_txt/pfm.tsv

## Paired_shift_view table
This table is a speed up table from three different tables. To create it, use the following query:
```SQL
CREATE TABLE paired_shift_view
    SELECT COUNT(*) as count,
           distance,
           consensus_motif_motif_id,
           experiment_experiment_id
    FROM summit
    INNER JOIN motif_pos ON motif_pos_motifpos_id = motifpos_id
    INNER JOIN peak ON peak_peak_id = peak_id GROUP BY distance,consensus_motif_motif_id,experiment_experiment_id;
```

## Venn_view table
Looks like it is created from other tables:
```SQL
CREATE TABLE venn_view 
      SELECT peak.experiment_experiment_id,
             peak.peak_id,
             summit.summit_id,
             motif_pos.motifpos_id,
             motif_pos.consensus_motif_motif_id
      FROM summit
      LEFT JOIN peak ON peak.peak_id = summit.peak_peak_id
      LEFT JOIN motif_pos ON motif_pos.motifpos_id = summit.motif_pos_motifpos_id;
```

# Creating the database
All the table creation process can be found in the table.ddl file. Do not forget: you need to delete the tables if you want to rerun this command!

In [ ]:
mysql -u root -p summitdb <tables.ddl

# Importing the tables
Because the name of the input files is the same as the database tables we can use mysqlimport to read all the files.

In [ ]:
mysqlimport -u root -p --local=1 summitdb antibody.tsv cell_lines.tsv consensus_motif.tsv genome.tsv anti2cons.tsv reference.tsv pfm.tsv experiment.tsv denovo_motif.tsv peak.tsv motif_pos.tsv average_deviation.tsv dbsnp.tsv summit.tsv

# Improvements
- Experiment sra_url too short
- denovo_motif table consensus_equence too short
- summit table height field is float, not integer